# 文本预处理
常见步骤
- 加载文本
- 文本拆分词元 `token`
- 建立词表，将 `token` 映射为数字索引
- 根据词表，将文本转化成数字索引序列

In [1]:
import collections
import re
from d2l import torch as d2l

# 读取数据集
数据集：$The Time Machine, by H.G.Well$ ，数据集并不大， `3w+` 个单词，现实中的文档可能会包含数十亿个单词。

这里忽略大写和标点符号

In [4]:
d2l.DATA_HUB["time_machine"] = ((d2l.DATA_URL) + "timemachine.txt",'090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    # 将数据集加载到文本行的列表中
    with open (d2l.download("time_machine","r")) as f:
        lines = f.readlines()
    # 正则表达式筛出标点，strip()筛出每行开头结尾的空白字符
    return [re.sub("[^A-Za-z]+"," ",line).strip().lower() for line in lines]

lines = read_time_machine()
print(f"文本总行数: {len(lines)}")
print(lines[0])
print(lines[10])

文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


# 词元化 `tokenize`
文本拆分成 `token` 

`token` 是文本的基本单位，分为 `word` 和 `char` 词元

In [7]:
def tokenize(lines,token="word"):
    '''将文本拆分成单词或字符词元'''
    if token == "word":
        return [line.split() for line in lines]
    if token == "char":
        return [list(line) for line in lines]
    else:
        print("Error: unknown token type: " + token)
        
tokens = tokenize(lines,token="word") # 列表嵌套
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


# 构建词表
- 语料：对去重后的文本中每一个唯一 `token` 的统计结果
- 词表：将语料库中的唯一 `token` 映射为数字索引，删去低频词以降低复杂度
- 特殊词：
 - 未知词元（被删去了或是不存在）：`<unk>`
 - 填充词元：`<pad>`
 - 序列开始词元：`<bos>`
 - 序列结束词元：`<eos>`

In [24]:
class Vocab:
    '''文本词表'''
    def __init__(self,tokens=None,min_freq=0,reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # 按freq排序
        counter = count_corpus(tokens)
        self._token_freqs = sorted(counter.items(),key=lambda x:x[1],reverse=True)
        # 未知词元的索引为0，存储在词表中
        self.idx_to_token = ["<unk>"] + reserved_tokens
        self.token_to_idx = {token:idx for idx,token in enumerate(self.idx_to_token)}
        
        for token,freq in self._token_freqs:
            if freq < min_freq:
                break
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                # token_to_idx的值是idx_to_token中token的索引，实现通过索引访问token
                self.token_to_idx[token] = len(self.idx_to_token) - 1 
            
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self,tokens):
        if not isinstance(tokens,(tuple,list)):
            return self.token_to_idx.get(tokens,self.unk)
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self,indices):
        if not isinstance(indices,(list,tuple)):
            return [self.idx_to_token[index] for index in indices]
        return [self.idx_to_token[index] for index in indices]
    
    def unk(self):
        # 未知词元索引是0
        return 0
    
    def token_freq(self):
        return self._token_freqs
    
def count_corpus(tokens):
    '''计算词元出现频率'''
    if len(tokens) == 0 or isinstance(tokens[0],list):
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [25]:
vocab = Vocab(tokens) # 构建词表
print(list(vocab.token_to_idx.items())[:10])


[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [29]:
for i in [0,10]:
    print(f"文本: ",tokens[i])
    print(f"索引: ",vocab[tokens[i]])

文本:  ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引:  [1, 19, 50, 40, 2183, 2184, 400]
文本:  ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引:  [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


# 功能整合
最终返回语料库 `corpus` 和词典 `vocab`

In [30]:
def load_corpus_time_machine(max_tokens=-1):
    '''返回时光机器的语料库和词典'''
    lines = read_time_machine()
    tokens = tokenize(lines,"word") # 以word为单位所以和书中不一样
    vocab = Vocab(tokens)
    corpus = [vocab[token] for line in tokens for token in line] # 展平
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus,vocab

corpus,vocab = load_corpus_time_machine()
len(corpus),len(vocab)

(32775, 4580)